<img align="center" width="12%" style="padding-right:10px;" src="Images/Ccom.png">

# Lab 4.0: Positioning Data<a href="https://piazza.com/class/jzvaaav18cf2j7"><img src="Images/help.png"  title="Ask questions on Piazza.com" align="right" width="10%" alt="Piazza.com\"></a><br><br> 

In this Notebook you will update the Position class from the Integrated Seabed Mapping Systems course Lab A 
assignment. The updates consists of adding a read method to parse the positioning data contained within HYPACK *.RAW* files.

For now the only positioning data that you will extract is that contained withing the **GGA** records

You have not yet encountered irregular ascii files i.e., files that do not have a predictable structure, this is a gentle introduction into it



In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import numpy as np

sys.path.append(os.getcwd())  # add the current folder to the list of paths where Python looks for modules 

___
## 4.0.0 Getting a copy of your position.py file in your project

We will add to the Position class that you developedin the course ESCI/OE 874. Normally we would just keep the one version of the class (this is the power of Object Oriented Programming), but in order to not create unintended consequences for the other class we will just copy the class file into this project.


<br><br>
<img width="3%" src="Images/logo.png">    
To avoid this click on the ePOM symbol (see above) on the top left of the page 
<br><br>
<img width="50%" src="Images/user_folder.png"> 
<br>
In your home folder click on the **New** button and select **Terminal** - You will be rewarded with a terminal
<br><br>
<img width="50%" src="Images/terminal.png"> 
<br>
In the terminal type `pwd` (Print Working Directory). In the image above you see that I am in my home directory (/home/jupyter-semmed). In your case it should show `/home/jupyter-yourusername`. If you are not there type `cd ~` which will get you to your home directory (`cd` is the change directory, `~` is short hand for your home directory)
<br><br>
Now that you are in your home directory type in `cp ESCI_OE_774_874_Public/Lab_A/mycode/position.py ESCI_872_Public/mycode/`. This will copy the file 'position.py' containing your definition for the class `Position` from your ESCI/OE 874 Lab A mycode folder to the mycode folder for this class. You can now safely edit the Position class without jeopardizing your work in the Integrated Seabed Mapping class. Note that there are **MUCH** better ways of doing this, which you will learn about at the end of the term.

You will know that you were successful if you can execute the code cell below


In [3]:
import import_ipynb
import os.path
import matplotlib as plt
from mycode.position import Position
from datetime import datetime, timezone

# get the absolute path to the current directory

abs_path=os.path.abspath(os.path.curdir)

# Instantiate a Position object and read data into it 

positions=Position()

### 4.0.1 Updating the Position Class Definition

There are a number of additional variables that we want to add to the position class in order to do this you should replace:

        # The geodetic coordinates - these are curvilinear so do not put them
        # in vectors, as that is a linear concept
        self.latitudes = list()
        self.longitudes = list()
        self.heights = list()

with:
        # The geodetic coordinates - these are curvilinear so do not put them
        # in vectors, as that is a linear concept
        self.latitudes = list()
        self.longitudes = list()
        self.heights = list()
        self.qualities = list()
        self.n_sats = list()
        self.hdops = list()
        self.separations = list()
        self.corr_ages = list()
        self.corr_stations = list()
        
This adds the ability to add relevant information for each position record namely a quality indicator which we will choose to be the same as the quality indicator used for **NMEA0183 GGA** messages. An indicator for the number of satellite vehicles used for the position solution. An indicator for the **HDOP** (Horizontal Dilution Of Precision). A parameter describing the orthogonal distances from the ellipsoid to the geoid. And finally a pair of parameters indicating the age of the last differential correction along with the source of that correction.

### 4.0.2 Creating a HYPACK Parser

Add the following code to Position class in the file position.py in the ESCI_872_Public/mycode folder.

    def read_hypack_raw_file(self, fullpath):
        
        # This function will currently only function provided that there are GGA sentences in the records.
        # You may update the function to include other positioning messages as well, but this is 
        # outside the scope of the class
                        
        # Check the File's existence
        if os.path.exists(fullpath):
            self.data_path = fullpath
            print('Opening GNSS data file:' + fullpath)
        else:  # Raise a meaningful error
            raise RuntimeError('Unable to locate the input file' + fullpath)
            
        # Open, read and close the file
        hypack_file = open(fullpath)
        hypack_content = hypack_file.read()
        hypack_file.close    

        # Split the file in lines
        hypack_records = hypack_content.splitlines()
        
        # Go through the header lines to find the date of the survey (not contained in the GGA records)
        
        lines_parsed=0
        for hypack_record in hypack_records:
            
            # Check for the time and date
            
            if hypack_record[:3].lower() == "tnd":
                hypack_datetime=datetime.strptime(hypack_record[4:23], "%H:%M:%S %m/%d/%Y")
                
                print("HYPACK RAW Header start time and date: " + hypack_datetime.ctime())
                
            # Keep track of the lines parsed
            lines_parsed+=1

            # Stop going through the records if the record starts with the string eoh (End Of Header)
            if hypack_record[:3].lower() == "eoh":
                break         
        
        # We are at the end of the header - start looking for the first GGA record and compare its time 
        # to the TND record
        # This is so that we can set the correct date
        
        # Keep track of the number of GGA records found
        
        num_gga_recs=0
        
        for hypack_record in hypack_records[lines_parsed:]:

            if hypack_record[19:22] == "GGA":
                gga_data=hypack_record.split()[3]
                gga_data=gga_data.split(',')
                
                # Determine the time of day from both the header and the GGA string
     
                gga_timedelta=timedelta(hours=int(gga_data[1][0:2]), \
                                        minutes = int(gga_data[1][2:4]), \
                                        seconds = int(gga_data[1][4:6]))
            
                hypack_timedelta=timedelta(hours = hypack_datetime.hour, \
                                           minutes = hypack_datetime.minute, \
                                           seconds = hypack_datetime.second)
            
                # If the hours are not the same we need to believe one or the other, which one?
                # We know from experience that the GGA time from Trimble receivers is indeed utc, 
                # whereas we also know that in older version of HYPACK the timestamp depended on the CPU time
                
                # Why do we care? We may end up with the timing for the wrong day if we are not careful! 
                # To know the correct date we need to take the date and time from the header and adjust it to UTC,
                # After that we know the time. If then the time difference between any succeeding GGA records 
                # is negative 
                # we know that we have just gone into a new day and need to update the datetime. 
                # Calculate the time difference in microseconds

                # Convert the HYPACK datetime to represent UTC time. We will make the ASSUMPTION that the 
                # header was created just before the first GGA record is received - this is _NOT_ fool 
                # proof (why?) 
                                
                hypack_datetime = hypack_datetime + \
                    timedelta( hours = round((gga_timedelta - hypack_timedelta).total_seconds() / 3600))
                
                print("HYPACK RAW Header start time and date in UTC: " + hypack_datetime.ctime())
                
                gga_datetime = hypack_datetime
                
                break
                
            # Keep track of where we are
                
            lines_parsed+=1
            
                
        # We are in the data file at the first GGA record - we know the date
  
        prev_gga_time = gga_timedelta
   
        
        for hypack_record in hypack_records[lines_parsed:]:
            if hypack_record[19:22] == "GGA":
                
                # Update the number of GGA records found
                
                num_gga_recs += 1
                
                # Get the GGA string and tokenize it
                
                gga_data=hypack_record.split()[3]
                gga_data=gga_data.split(',')
                
                # Determine the time of day from both the header and the GGA string
     
                gga_timedelta=timedelta(hours=int(gga_data[1][0:2]), \
                                        minutes = int(gga_data[1][2:4]), \
                                        seconds = int(gga_data[1][4:6]))
            
                # Check whether we rolled into a new day - we assume time is always increasing
                
                if gga_timedelta < prev_gga_time:
                    # We have reached the next day - update the date
                    
                    gga_datetime += timedelta(days=1)
                    
                    print( "Passed midnight, updating date to: "+str(gga_datetime.date()))
                    
                # Create the time
                
                time=gga_datetime.replace(hour=int(gga_data[1][0:2]), \
                                        minute = int(gga_data[1][2:4]), \
                                        second = int(gga_data[1][4:6]))

                # Add the time object to the list of times

                pass
                # Insert code here ***
                
                # Parse the latitude

                if gga_data[3].lower() == "n":
                    self.latitudes.append( float(gga_data[2][0:2])+float(gga_data[2][3:])/60.)
                else:
                    pass
                    # Insert code here ***               

                # Parse the longitudes - NOTE that longitude used 3 digits before the decimal

                if gga_data[5].lower == "w":
                    pass
                    # Insert code here ***
                else:
                    pass
                    # Insert code here ***               

                # Parse the GNSS Quality indicator
                
                pass
                # Insert code here ***

                # Parse the number of GNSS satellites used for the solution
                
                pass
                # Insert code here ***

                # Parse the HDOP Quality indicator
                
                pass
                # Insert code here ***

                # Parse the orthometric height 
                
                pass
                # Insert code here ***
                
                # Generate an error if the units of the orthometric height is not meters
                                   
                if gga_data[10].lower() != "m":
                    raise RuntimeError('Orthomeric height units are not meters!')                
                
                # Parse the geoid ellipsoid separation
                
                pass
                # Insert code here ***
                                   
                if gga_data[12].lower() != "m":
                    raise RuntimeError('Orthomeric height units are not meters!') 
                    
                # If there is more data then parse it
                
                if gga_data[13] != "":
                    self.corr_ages.append(float(gga_data[13]))
                    self.corr_stations.append(float(gga_data[14]))
                    
                # For now, ignore the checksum (this would become a computer science assignment
                
                # Make sure to update the previous gga time
                    
                pass
                # Insert code here ***

        # Set the reference ellipsoid to WGS84

        self.metadata["ellipsoid_name"] = "WGS84"
        self.metadata["geoid_name"] = "EGM08"
        self.metadata["height_relative_to"] = "geoid"

        # Let the user know how many GGA records there were in the file
        print("HYPACK RAW file contains: " + str(num_gga_recs) + " GGA records")
    

In [4]:
positions.read_hypack_raw_file(abs_path+'/Data/000_1029.RAW')

Opening GNSS data file:/home/jupyter-semme/ESCI_872/Data/000_1029.RAW
HYPACK RAW Header start time and date: Tue Jul  3 10:29:07 2012
HYPACK RAW Header start time and date in UTC: Tue Jul  3 14:29:07 2012
HYPACK RAW file contains: 58 GGA records


## 4.1 Let us walk through the code:

You will notice that we read the code in three parts: 1) The header, 2) The data until the first GGA record, and 3) the data from the first GGA record

This seems cumbersome - the reason why has to do with the information contained within the GGA record

### 4.1.0 GGA Records

Research the trimble GGA record - is the epoch data complete (discuss on Piazza)

### 4.1.1 TND Records

Look at the TND record in the HYPACK file header (lines up to EOH) - can you use this data to complete the epochs (discuss on Piazza)

### 4.1.2 GGA and TND records

There appears to be a mismatch between the times in the GGA and the TND records - what is the problem? (discuss on Piazza)

### 4.1.3 Reconciling the Epochs

Analyze the code given to you - how does this deal with the issue? (discuss on Piazza)

### 4.1.4 Is the reconciliation complete?

Note that the first GGA record is received after the file header is created i.e., the time is different. Assuming that both the CPU time and the GGA time are correct, just relative to a different time zone, is there still the potential for getting the wrong date with the code as is provided here?

If so when do you think problems may occur?

### 4.1.5 Complete the HYPACK reading method 

Replace the sections marked **"# Insert code here** With actual code. Note that there is the keyword `pass` preceding most of these sections. These are there so that the code can be executed without error as it is given to you. Once you have inserted the necessary code in a section you should be able to remove the `pass` preceding it. Once you are done there should not be any occurrence of the `pass` keyword in the `read_hypack_raw_file` method

### 4.1.6 Create a map of the data read

Produce a map using the *position.draw()* method in the code cell below - note that this will *only* work properly once you have completed the `read_hypack_raw_file` method


In [8]:
positions.draw()

Drawing Positioning Data


ValueError: max() arg is an empty sequence

<img align="left" width="5%" style="padding-right:10px;" src="Images/email.png">

*For issues or suggestions related to this notebook that should not be addressed on Piazza, write to: semmed@ccom.unh.edu*